# Plan


- Introduction about me/Massive (1m)
    - ME
        - Work
        - My fav games
    - Massive:
        - Ubisoft studio
        - MSV Games
        - Game Intelligence Department
- Funny hook to MLOps (3m)
  - Common ML issue / things that can go really wrong
      - Microsoft Bot
          - Show what happened
          - If they had a monitoring system tracking as KPI AI racism, they would have noticed things going wrong before it was too late
          - This shows there are two important points:
              - Live monitoring
              - Set your KPI right! Monitoring the wrong things is not really that useful
      - In a data driven world, do you want to leave your bot free to act?
          - Not all AIs results are as visible,easy to grasp as the Microsoft Bot.
  - Questions to the audience:
      - Who has deployed an ML app to production?
      - Who has tracked/monitored an ML app?
- Agenda Slide (1m)
- Going through the presentation (See Agenda below)
- Conlusions (1m)
- Examples on how to contribute to these projects (1m)
- How to contact me
    - GitHub
    - Linkedin
    - PS handle
    - Ubiconnect handle

# Agenda


- Traditional ML development VS MLOps driven ML development (3 mins)
    - Why is DevOps good in Software
    - BUT ML is a different case
        - Explain multidimensional ML challenge vs software engineering (code VS data)
    
- Key concepts in MLops (5 mins)
    - Waterfall cycle vs Agile cycle
    - Focus on Monitoring

    
- MLFlow (3 mins)
    - What is MLFlow?
        - Tracking
        - Projects
        - Model
        - Registry
    - Getting started: https://mlflow.org/docs/latest/quickstart.html
    
- Funny game intermission
    - funny video about some game stuff
    - OR when switching between screens show desktop background being muffins vs cihuaua / sheepdog or mop (joke: yes I keep training myself every day, you don't want to get outdated)
        
- MLFlow for data and model tracking in Python (8 mins)
Check below notes

- Grafana for human friendly monitoring of a live ML application (5 mins)
    - Python has many great options 
        - for custom dashboards (Panel, Dash, pyscript can make it run directly in browser)
        - Local notebook, but not live
    - BUT Grafana is a actually an out-of-the box tool that can be used by everyone in your team very easily.
    - How to get an instance of Grafana running
    - Grafana queries:
        - Grafana to DB
        - Grafana to Rest API
    - Grafana interactive dashboards
    - Grafana alerts:
        - examples
    - Grafana resources:
        - List of Dashboards: https://grafana.com/grafana/dashboards/

- Common Pitfalls and tricks (2m)
    - Personal reference: worked for 9 months on a project, finally your data is clean, DNN is hyper-tuned, cross-validated, trained and deployed. Done! But are you really?
    - Figure out your KPIs!
    
- Main Takeways
    - For ML Engineers/ Data Scientists When starting an ML project ask yourself this question: Is this going to production? Is this going to automate process/drive decisions?
        - If no, just hack away in your notebook. Be my guest!
        - If yes, start with an MLOps mindset. Information are out there, and platforms are starting to become more stable.
    - Are you an ML Product Owner, project manager, team lead or others?
        - Things are moving fast. Data Engineers, data scientists and ML Engineers are becoming more and more specialized, and with different interests. Show image. Consider hiring different people for different tasks!
        - ML takes time! Plan in advance and consider your cycle end to end!

# MLOps monitoring notes

## Flow
    A typical continuous monitoring process consists of the following steps:
    1. A sample of the request-response payloads is captured and stored in the serving logs store.
    2. The monitoring engine periodically loads the latest inference logs, generates a schema, and computes statistics for the serving data.
    3. The monitoring engine compares the generated schema to a reference schema to identify schema skews,
    4. If the true labels (ground truth) for the serving data are available, the monitoring engine uses it to evaluate the model predictive effectiveness in hindsight on the serving data.
    5. If anomalies are identified, or if the model’s performance is decaying, alerts can be pushed through various channels (for example, email or chat) to notify the owners to examine the model or to trigger a new retraining cycle. 



## Continuous Monitoring
Typical assets produced in this
process include the following:
• Anomalies detected in serving data
during drift detection
• Evaluation metrics produced from
continuous evaluation
Core MLOps capabilities:
• Dataset & feature repository
• Model monitoring
• ML metadata & artifact repository